# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output

!pip3 install pyprind

clear_output()

# Instructions

* Clone the notebook to your drive. 
* The notebook has to be submitted with the subject as "CFI_AI_DF_PM_23-24_\<your name\>_\<roll number\>". 

* If you have any queries, you can reach out to the core team:

| Name | Phone Number |
| :-- | :-- |
| Karthick Krishna | 7338857571|
| Tharun Anand | 7904225519 |


# Coding Questions



### Question 1

Implement, from scratch, CNN architecture commonly used for [Image Classification](https://www.thinkautomation.com/eli5/eli5-what-is-image-classification-in-deep-learning/), namely, [VGG19](https://medium.com/mlearning-ai/image-detection-using-convolutional-neural-networks-89c9e21fffa3), [LeNet](https://towardsdatascience.com/understanding-and-implementing-lenet-5-cnn-architecture-deep-learning-a2d531ebc342), [AlexNet](https://towardsdatascience.com/alexnet-the-architecture-that-challenged-cnns-e406d5297951), [ResNet](https://towardsdatascience.com/an-overview-of-resnet-and-its-variants-5281e2f56035), etc., using [PyTorch](https://pytorch.org/) to classify the images of the CIFAR10 dataset. 

**Hint:** You have to implement the `torch.nn.module` class for the models. (Brownie Points for implementing all the named networks.)

**Caution:** Do not copy pre-existing implementations blindly. 


A template has been provided to help you. 

* You may or may not edit the `None` fields. There is no need to change anything else.
* You might or might not have to add additional lines other than what is provided. 

In [ ]:
# Downloading and Preparing the Dataset

!gdown --id 1oYnD7Izl3LVVzjEMyLxLklX30TKWHgGG
!unzip /content/cifar-10.zip
!rm -rf /content/cifar-10.zip
!mv /content/cifar-10/sample_submission.csv /content/cifar-10/test_labels.csv

clear_output()

In [ ]:
# Imports

import torch
import torchvision

from PIL import Image

import pandas
import numpy
from sklearn import preprocessing
import matplotlib

import os
import pyprind

PATH = "https://drive.google.com/drive/folders/1C4n9hGzzxiypA4s6p2NOZK4_JkAfPw43"

In [ ]:
from torchvision.transforms.transforms import Normalize

class CreateDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, mode='train'):
        self.root_dir = root_dir
        self.mode = mode

        self.entry = pandas.read_csv(os.path.join(self.root_dir, f'{self.mode}_labels.csv'))
        self.encoder = self._process_()
        self.entry['label'] = self.encoder.transform(self.entry['label'])

        self.transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ]
        )

    def _process_(self):
        data = pandas.read_csv(os.path.join(self.root_dir, f'{self.mode}_labels.csv'))
        encoder = preprocessing.LabelEncoder()
        encoder.fit(data['label'])
        return encoder

    def __getitem__(self, index):
        data = self.entry.loc[index, 'id']
        image = Image.open(os.path.join(self.root_dir, f'{self.mode}', f'{data}.png')) 
        image = self.transform(image)
        label = self.entry.loc[index, 'label'] 
        return image, label
        

    def __len__(self):
        return len(self.entry)

#CNN Architectures

CNN architectures are all some of the Computer vision neural networks generated by several top organisations while participating in the ImageNet competitions.These architectures are each developed with some innovative treat which helps us to generate more and more accurate results.

##ResNet-
Residual Network (ResNet) is a Convolutional Neural Network (CNN) architecture that overcame the “vanishing gradient” problem, making it possible to construct networks with up to thousands of convolutional layers, which outperform shallower networks.We are achieving this by adding a common Residual layer in evry ResNet layer so that the gradient can easily pass through.

1. The below is the Residual Block or the repetetive block which repoeats in every layer of the ResNet layers

In [ ]:
class ResidualBlock(torch.nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(out_channels)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = torch.nn.BatchNorm2d(out_channels)
        self.conv3 = torch.nn.Conv2d(out_channels, self.expansion*out_channels, kernel_size=1, bias=False)
        self.bn3 = torch.nn.BatchNorm2d(self.expansion*out_channels)

        self.shortcut = torch.nn.Sequential()
        if stride != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels, self.expansion*out_channels,kernel_size=1, stride=stride, bias=False),
                torch.nn.BatchNorm2d(self.expansion*out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out


2. Given below is the ResNet Layer which is developed with the Residual block as one of the added attachments in every layer.

In [ ]:
import torch.nn.functional as F

class ResNet(torch.nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(64)
        self.relu = torch.nn.ReLU()
        self.layer1 = self._make_layer(ResidualBlock, 64, 3, stride=1)
        self.layer2 = self._make_layer(ResidualBlock, 128, 4, stride=2)
        self.layer3 = self._make_layer(ResidualBlock, 256, 6, stride=2)
        self.layer4 = self._make_layer(ResidualBlock, 512, 3, stride=2)
        self.linear = torch.nn.Linear(512*ResidualBlock.expansion, num_classes)

    def _make_layer(self, ResidualBlock, out_channels,num_blocks ,stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(ResidualBlock(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * ResidualBlock.expansion
        return torch.nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [ ]:
resnet = ResNet(ResidualBlock)

##VGGNet-
VGG19 is an advanced CNN with pre-trained layers and a great understanding of what defines an image in terms of shape, color, and structure. VGG19 is very deep and has been trained on millions of diverse images with complex classification tasks.Below VGG19 has several layers whixh are in a pattern where there is a repetition of 3 layers-
1. `Conv2d`-Convolutional layer
2. `BatchNorm`- Batch Normalisation layer which brings the matrices within range when deviated
3. `ReLU` - Rectified Linear Unit Layer

Another is repetition of `Maxpooling` Layer - which is generally used for reducing the dimension of the feature maps.

In [ ]:
VGG19 = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],

class VGG(torch.nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = self._make_layers(VGG19)
        self.classifier = torch.nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, vgglist):
        layers = []
        in_channels = 3
        for x in vgglist:
            if x == 'M':
                layers += [torch.nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [torch.nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           torch.nn.BatchNorm2d(x),
                           torch.nn.ReLU(inplace=True)]
                in_channels = x
        layers += [torch.nn.AvgPool2d(kernel_size=1, stride=1)]
        return torch.nn.Sequential(layers)

##AlexNet-
AlexNet. The architecture consists of eight layers: five convolutional layers and three fully-connected layers and also a flatten layer.

1. `ReLU Nonlinearity`- AlexNet uses Rectified Linear Units (ReLU) instead of the tanh function, which was standard at the time.
2. `Multiple GPUs`- Back in the day, GPUs were still rolling around with 3 gigabytes of memory (nowadays those kinds of memory would be rookie numbers).
3. `Overlapping Pooling`- CNNs traditionally “pool” outputs of neighboring groups of neurons with no overlapping. However, when the authors introduced overlap.

In [ ]:
class AlexNet(torch.nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(64, 192, kernel_size=5, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(192, 384, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(384, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = torch.nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


##LeNet-
LeNet-5 CNN architecture is made up of 7 layers. The layer composition consists of 3 convolutional layers, 2 subsampling layers and 2 fully connected layers.

In [ ]:
class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, kernel_size=5)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1   = torch.nn.Linear(16*5*5, 120)
        self.fc2   = torch.nn.Linear(120, 84)
        self.fc3   = torch.nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

#Training the Model

In [ ]:
class Trainer():
    def __init__(self, data, model):
        self.data = data
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.trainloader, self.validloader, self.testloader = self.get_iterator(self.data)
        
        self.model = self.get_model(model).to(self.device)
        self.criterion = self.get_criterion().to(self.device)
        self.optimizer = self.get_optimizer()
        self.train_loss = []
        self.train_metrics = []
        self.valid_loss = []
        self.valid_metrics = []
        self.epochs = 10

    def get_iterator(self, data):
        train, valid, test = data
        trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, drop_last=True) 
        validloader = torch.utils.data.DataLoader(valid, batch_size=128, shuffle=False, drop_last=True) 
        testloader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=False) 
        return trainloader, validloader, testloader

    def get_criterion(self):
        return torch.nn.CrossEntropyLoss()
    
    def get_optimizer(self):
        return torch.optim.RMSprop(self.model.parameters(), lr = 0.0001)

    def get_model(self, model):
        return model

    def save(self, epoch):
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            }, os.path.join(PATH, "model.pth"))
        
    def load(self):
        if os.path.exists(os.path.join(PATH, "model.pth")):
            checkpoints = torch.load(os.path.join(self.args.checkpoint, "model.pth"), map_location=self.device)
            self.model.load_state_dict(checkpoints['model_state_dict'])
            self.optimizer.load_state_dict(checkpoints['optimizer_state_dict'])

    def train(self):
        epoch_loss = 0
        epoch_metrics = {}

        self.model.train()

        with torch.autograd.set_detect_anomaly(True):
            bar = pyprind.ProgBar(len(self.trainloader), bar_char='█')
            for index, (image, label) in enumerate(self.trainloader):
                image = image.to(self.device)
                label = label.to(self.device)
                image = image.squeeze(0)

                self.optimizer.zero_grad()                
                output = self.model.forward(torch.tensor(image.clone().detach()))
                #Evaluating the loss function
                loss = self.criterion(output,label)
                loss.backward()
                epoch_loss += loss.item()
                #Back propagation - gradient descent
                self.optimizer.step()
                bar.update()

            epoch_loss = epoch_loss/128 #Dividing by the batch size

        return epoch_loss, epoch_metrics

    def evaluate(self):
        epoch_loss = 0
        epoch_metrics = {}

        with torch.autograd.set_detect_anomaly(True):
            bar = pyprind.ProgBar(len(self.validloader), bar_char='█')
            for index, (image, label) in enumerate(self.validloader):
                image = image.to(self.device)
                label = label.to(self.device)                
                output = self.model(torch.tensor(image.squeeze(0).clone().detach()))
                loss = self.criterion(output, label)
                epoch_loss += loss.item()
                bar.update()

        return epoch_loss, epoch_metrics

    def test(self):

        self.model.eval()

        outputs = torch.empty([0,]).to(self.device)

        with torch.autograd.set_detect_anomaly(True):
            bar = pyprind.ProgBar(len(self.testloader), bar_char='█')
            for index, (image, label) in enumerate(self.testloader):
                image = image.to(self.device)
                label = label.to(self.device)
                output = self.model(torch.tensor(image.squeeze(0).clone().detach())).to(self.device)
                outputs = torch.cat([outputs, output]).to(self.device)

                bar.update()

        return outputs
    
    def fit(self):

        for epoch in range(1, self.epochs+1, 1):

            epoch_train_loss, epoch_train_metrics = self.train()

            self.train_loss.append(epoch_train_loss)
            self.train_metrics.append(epoch_train_metrics)

            epoch_valid_loss, epoch_valid_metrics = self.evaluate()
            
            self.valid_loss.append(epoch_valid_loss)
            self.valid_metrics.append(epoch_valid_metrics) 

            print(f'Epoch {epoch}/{self.epochs+1}: Train Loss = {epoch_train_loss} | Validation Loss = {epoch_valid_loss}')

In [ ]:
train_data = CreateDataset(root_dir="/content/cifar-10", mode='train') 
train_data, valid_data = torch.utils.data.random_split(train_data, [len(train_data)-len(train_data)//10, len(train_data)//10])
test_data = CreateDataset(root_dir="/content/cifar-10", mode='test') 
data = (train_data, valid_data, test_data)

trainer = Trainer(data, resnet)
trainer.fit()

outputs = trainer.test()

<ipython-input-12-049971e97a8f>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = self.model.forward(torch.tensor(image.clone().detach()))
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:06
<ipython-input-12-049971e97a8f>:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = self.model(torch.tensor(image.squeeze(0).clone().detach()))
0% [██████████████████████████████] 100% | ETA: 00:00:00

Epoch 1/11: Train Loss = 4.0931671699509025 | Validation Loss = 47.18881332874298



Total time elapsed: 00:00:07
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:03
0% [██████████████████████████████] 100% | ETA: 00:00:00

Epoch 2/11: Train Loss = 2.7422315306030214 | Validation Loss = 36.314328372478485



Total time elapsed: 00:00:07
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:03
0% [██████████████████████████████] 100% | ETA: 00:00:00

...


Total time elapsed: 00:00:07
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:03
0% [██████████████████████████████] 100% | ETA: 00:00:00

Epoch 4/11: Train Loss = 1.4095873963087797 | Validation Loss = 27.560781240463257



Total time elapsed: 00:00:07
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:01
0% [██████████████████████████████] 100% | ETA: 00:00:00

Epoch 5/11: Train Loss = 0.9421912017278373 | Validation Loss = 29.563781440258026



Total time elapsed: 00:00:07
0% [█                             ] 100% | ETA: 00:02:52

##Question 2 (Optional)

EfficientNet is a convolutional neural network architecture and scaling method that uniformly scales all depth/width/resolution dimensions using a compound coefficient. Use [Transfer Learning](https://machinelearningmastery.com/transfer-learning-for-deep-learning/) to extract the layers of EfficientNet and perform image classification for the same dataset.

In [ ]:
import torch

In [ ]:
from IPython.display import clear_output

!pip3 install pyprind

clear_output()

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet 

In [ ]:
# Load the EfficientNet model
enet = EfficientNet.from_pretrained('efficientnet-b0', num_classes=10)

In [ ]:
# Imports

import torch
import torchvision

from PIL import Image

import pandas
import numpy
from sklearn import preprocessing
import matplotlib

import os
import pyprind

PATH = "https://drive.google.com/drive/folders/1C4n9hGzzxiypA4s6p2NOZK4_JkAfPw43"

In [ ]:
from torchvision.transforms.transforms import Normalize

class CreateDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, mode='train'):
        self.root_dir = root_dir
        self.mode = mode

        self.entry = pandas.read_csv(os.path.join(self.root_dir, f'{self.mode}_labels.csv'))
        self.encoder = self._process_()
        self.entry['label'] = self.encoder.transform(self.entry['label'])

        self.transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ]
        )

    def _process_(self):
        data = pandas.read_csv(os.path.join(self.root_dir, f'{self.mode}_labels.csv'))
        encoder = preprocessing.LabelEncoder()
        encoder.fit(data['label'])
        return encoder

    def __getitem__(self, index):
        data = self.entry.loc[index, 'id']
        image = Image.open(os.path.join(self.root_dir, f'{self.mode}', f'{data}.png')) 
        image = self.transform(image)
        label = self.entry.loc[index, 'label'] 
        return image, label
        

    def __len__(self):
        return len(self.entry)

In [ ]:
for param in enet.parameters():
    param.requires_grad = False

In [ ]:
enet.classifier = torch.nn.Sequential(
    torch.nn.Linear(1280, 512),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(512, 10)
)

In [ ]:
class Trainer():
    def __init__(self, data, model):
        self.data = data
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.trainloader, self.validloader, self.testloader = self.get_iterator(self.data)
        
        self.model = self.get_model(model).to(self.device)
        self.criterion = self.get_criterion().to(self.device)
        self.optimizer = self.get_optimizer()

        self.train_loss = []
        self.train_metrics = []
        self.valid_loss = []
        self.valid_metrics = []

        self.epochs = 1

    def get_iterator(self, data):
        train, valid, test = data
        trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, drop_last=True) 
        validloader = torch.utils.data.DataLoader(valid, batch_size=128, shuffle=False, drop_last=True) 
        testloader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=False) 
        return trainloader, validloader, testloader

    def get_criterion(self):
        return torch.nn.CrossEntropyLoss()
    
    def get_optimizer(self):
        return torch.optim.Adam(self.model.parameters())

    def get_model(self, model):
        return model

    def save(self, epoch):
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            }, os.path.join(PATH, "model.pth"))
        
    def load(self):
        if os.path.exists(os.path.join(PATH, "model.pth")):
            checkpoints = torch.load(os.path.join(self.args.checkpoint, "model.pth"), map_location=self.device)
            self.model.load_state_dict(checkpoints['model_state_dict'])
            self.optimizer.load_state_dict(checkpoints['optimizer_state_dict'])

    def train(self):
        epoch_loss = 0
        epoch_metrics = {}

        self.model.train()

        with torch.autograd.set_detect_anomaly(True):
            bar = pyprind.ProgBar(len(self.trainloader), bar_char='█')
            for index, (image, label) in enumerate(self.trainloader):
                image = image.to(self.device)
                label = label.to(self.device)
                image = image.squeeze(0)

                self.optimizer.zero_grad()                
                output = self.model.forward(torch.tensor(image.clone().detach()))
                #Evaluating the loss function
                loss = self.criterion(output,label)
                epoch_loss += loss.item()
                #Back propagation - gradient descent
                self.optimizer.step()
                bar.update()

        return epoch_loss, epoch_metrics

    def evaluate(self):
        epoch_loss = 0
        epoch_metrics = {}

        with torch.autograd.set_detect_anomaly(True):
            bar = pyprind.ProgBar(len(self.validloader), bar_char='█')
            for index, (image, label) in enumerate(self.validloader):
                image = image.to(self.device)
                label = label.to(self.device)                
                output = self.model(torch.tensor(image.squeeze(0).clone().detach()))
                loss = self.criterion(output, label)
                epoch_loss += loss.item()
                bar.update()

        return epoch_loss, epoch_metrics

    def test(self):

        self.model.eval()

        outputs = torch.empty([0,]).to(self.device)

        with torch.autograd.set_detect_anomaly(True):
            bar = pyprind.ProgBar(len(self.testloader), bar_char='█')
            for index, (image, label) in enumerate(self.testloader):
                image = image.to(self.device)
                label = label.to(self.device)
                output = self.model(torch.tensor(image.squeeze(0).clone().detach())).to(self.device)
                outputs = torch.cat(torch.tensor(outputs), output).to(self.device)

                bar.update()

        return outputs
    
    def fit(self):

        for epoch in range(1, self.epochs+1, 1):

            epoch_train_loss, epoch_train_metrics = self.train()

            self.train_loss.append(epoch_train_loss)
            self.train_metrics.append(epoch_train_metrics)

            epoch_valid_loss, epoch_valid_metrics = self.evaluate()
            
            self.valid_loss.append(epoch_valid_loss)
            self.valid_metrics.append(epoch_valid_metrics) 

            print(f'Epoch {epoch}/{self.epochs+1}: Train Loss = {epoch_train_loss} | Validation Loss = {epoch_valid_loss}')

In [ ]:
train_data = CreateDataset(root_dir="/content/cifar-10", mode='train') 
train_data, valid_data = torch.utils.data.random_split(train_data, [len(train_data)-len(train_data)//10, len(train_data)//10])
test_data = CreateDataset(root_dir="/content/cifar-10", mode='test') 
data = (train_data, valid_data, test_data)

trainer = Trainer(data, enet)
trainer.fit()

outputs = trainer.test()